# Create Daily Dataset

this notebook creates a daily dataset, using the dataset with hourly data but using new hourly evaporation data from meteo suisse 

In [221]:
import pandas as pd

df = pd.read_csv("../Data/data_reduced_variables.csv")

eva =pd.read_csv("../data/order_107685_data.txt", sep=";")


# convert data_reduced_variables to daily format

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60754 entries, 0 to 60753
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 60754 non-null  object 
 1   suction_tension (20cm)   60754 non-null  float64
 2   suction_tension (40cm)   60754 non-null  float64
 3   suction_tension (70cm)   60754 non-null  float64
 4   water_temperature        60754 non-null  float64
 5   barometric_pressure_qfe  60754 non-null  float64
 6   dew_point                60754 non-null  float64
 7   water_level              60754 non-null  float64
 8   precipitation (mm)       60754 non-null  float64
 9   change_rate              60754 non-null  float64
 10  inflow_linth (m3/s)      60754 non-null  float64
 11  outflow_limmat (m3/s)    60754 non-null  float64
 12  evaporation              60754 non-null  float64
dtypes: float64(12), object(1)
memory usage: 6.0+ MB


In [223]:
# import pytz package for handling timezones
import pytz
# convert DateTime column into datetime format
df["datetime"] = pd.to_datetime(df['datetime'], utc=True)
# function that converts datetime column into CET timezone and accounts for Daylight saving
def add_timezone(df):
    timeZone = pytz.timezone("CET")
    df['datetime'] = df['datetime'].dt.tz_convert(timeZone)
    return df

add_timezone(df)

# set DateTime as index
df.set_index("datetime", inplace=True)

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60754 entries, 2012-01-01 01:00:00+01:00 to 2019-11-27 05:00:00+01:00
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   suction_tension (20cm)   60754 non-null  float64
 1   suction_tension (40cm)   60754 non-null  float64
 2   suction_tension (70cm)   60754 non-null  float64
 3   water_temperature        60754 non-null  float64
 4   barometric_pressure_qfe  60754 non-null  float64
 5   dew_point                60754 non-null  float64
 6   water_level              60754 non-null  float64
 7   precipitation (mm)       60754 non-null  float64
 8   change_rate              60754 non-null  float64
 9   inflow_linth (m3/s)      60754 non-null  float64
 10  outflow_limmat (m3/s)    60754 non-null  float64
 11  evaporation              60754 non-null  float64
dtypes: float64(12)
memory usage: 6.0 MB


In [225]:
# resampling dataframe to daily data by subsetting original dataframe into dataframes where we have to sum the hourly data (dfDS) 
# and dataframes where we have to take the mean of hourly data
dfDS = df[["precipitation (mm)"]].resample("D").sum()

dfDM = df[['suction_tension (20cm)', 'suction_tension (40cm)',
       'suction_tension (70cm)', 'water_temperature',
       'barometric_pressure_qfe', 'dew_point',
       'water_level', "inflow_linth (m3/s)", "outflow_limmat (m3/s)"]].resample("D").mean().interpolate(option='spline')


In [226]:
dfDM.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2888 entries, 2012-01-01 00:00:00+01:00 to 2019-11-27 00:00:00+01:00
Freq: D
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   suction_tension (20cm)   2888 non-null   float64
 1   suction_tension (40cm)   2888 non-null   float64
 2   suction_tension (70cm)   2888 non-null   float64
 3   water_temperature        2888 non-null   float64
 4   barometric_pressure_qfe  2888 non-null   float64
 5   dew_point                2888 non-null   float64
 6   water_level              2888 non-null   float64
 7   inflow_linth (m3/s)      2888 non-null   float64
 8   outflow_limmat (m3/s)    2888 non-null   float64
dtypes: float64(9)
memory usage: 225.6 KB


In [227]:
# merge the two datasets again 
dfD = pd.merge(dfDM, dfDS, how = 'inner', on=dfDM.index)#right_index = True, left_index = True)
# set datetime as index
dfD.set_index("key_0", inplace=True)
# change index name
dfD.index.names = ['datetime']
# create variable for change rate of water level since we didn´t want to use the one with hourly data and convert it to daily data
dfD["change_rate"] = dfD["water_level"].pct_change()
dfD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2888 entries, 2012-01-01 00:00:00+01:00 to 2019-11-27 00:00:00+01:00
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   suction_tension (20cm)   2888 non-null   float64
 1   suction_tension (40cm)   2888 non-null   float64
 2   suction_tension (70cm)   2888 non-null   float64
 3   water_temperature        2888 non-null   float64
 4   barometric_pressure_qfe  2888 non-null   float64
 5   dew_point                2888 non-null   float64
 6   water_level              2888 non-null   float64
 7   inflow_linth (m3/s)      2888 non-null   float64
 8   outflow_limmat (m3/s)    2888 non-null   float64
 9   precipitation (mm)       2888 non-null   float64
 10  change_rate              2887 non-null   float64
dtypes: float64(11)
memory usage: 270.8 KB


In [228]:
dfD.loc[dfD["water_temperature"].isna()]

,suction_tension (20cm),suction_tension (40cm),suction_tension (70cm),water_temperature,barometric_pressure_qfe,dew_point,water_level,inflow_linth (m3/s),outflow_limmat (m3/s),precipitation (mm),change_rate
datetime,,,,,,,,,,,


# clean and prepare evaporation dataset

In [229]:
# drop column we don´t need
eva.drop(columns="erefaod0", inplace=True)
# select only observation from one station (Wädenswil)
eva = eva.loc[eva["stn"]=="WAE"]
# drop the column for station names 
eva.drop(columns = "stn", inplace = True)
# rename coumns
eva.rename(columns={"time": "datetime","ets150d0": "Evapotranspiration (mm)"}, inplace = True)

In [230]:
eva

,datetime,Evapotranspiration (mm)
0,20120101,0.03
1,20120102,0.04
2,20120103,0.25
3,20120104,0.11
4,20120105,0.06
...,...,...
3971,20221115,0.14
3972,20221116,0.33
3973,20221117,0.31
3974,20221118,0.12


In [231]:
# import pytz package for handling timezones
import pytz
# convert DateTime column into datetime format
eva["datetime"] = pd.to_datetime(eva['datetime'], utc=True)
# function that converts datetime column into CET timezone and accounts for Daylight saving
def add_timezone(df):
    timeZone = pytz.timezone("CET")
    df['datetime'] = df['datetime'].dt.tz_convert(timeZone)
    return df

add_timezone(eva)

# set DateTime as index
eva.set_index("datetime", inplace=True)

In [232]:
eva#.info()

,Evapotranspiration (mm)
datetime,
2012-01-01 01:00:00+01:00,0.03
2012-01-02 01:00:00+01:00,0.04
2012-01-03 01:00:00+01:00,0.25
2012-01-04 01:00:00+01:00,0.11
2012-01-05 01:00:00+01:00,0.06
...,...
2022-11-15 01:00:00+01:00,0.14
2022-11-16 01:00:00+01:00,0.33
2022-11-17 01:00:00+01:00,0.31


# Merge the two Datasets


In [233]:
# merge the dfD dataframe with the evaporation dataframe on the index
merged_df = pd.merge(dfD, eva, how = 'left', right_index = True, left_index = True)

merged_df

,suction_tension (20cm),suction_tension (40cm),suction_tension (70cm),water_temperature,barometric_pressure_qfe,dew_point,water_level,inflow_linth (m3/s),outflow_limmat (m3/s),precipitation (mm),change_rate,Evapotranspiration (mm)
datetime,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,1.743478,1.356522,1.504348,6.023188,972.383333,6.369565,405.911957,32.034087,141.302696,0.1,NaN,NaN
2012-01-02 00:00:00+01:00,2.085417,1.595833,1.770833,6.048611,969.075000,5.038889,405.913472,35.996333,121.419250,8.0,0.000004,NaN
2012-01-03 00:00:00+01:00,2.179167,1.700000,1.641667,5.959722,976.445833,0.961111,405.923819,38.921458,121.583167,1.1,0.000025,NaN
2012-01-04 00:00:00+01:00,1.745833,1.375000,1.606250,5.915972,974.427778,1.297917,405.926250,39.451292,121.705458,9.9,0.000006,NaN
2012-01-05 00:00:00+01:00,1.435417,1.189583,1.543750,5.731250,961.188194,1.251389,405.925625,41.358000,138.889167,21.0,-0.000002,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-23 00:00:00+01:00,4.255208,3.026875,3.247500,10.315278,949.419444,3.397917,405.948889,45.610375,73.740125,0.0,-0.000027,NaN
2019-11-24 00:00:00+01:00,4.528125,3.240625,3.990000,10.234028,958.042361,4.770139,405.942014,43.226625,72.923125,0.0,-0.000017,NaN
2019-11-25 00:00:00+01:00,4.723958,3.411458,4.289792,10.204861,962.402917,5.890139,405.941597,41.914958,64.105333,0.0,-0.000001,NaN


In [235]:
# safe dataframe as csv
merged_df.to_csv("../data/daily_data.csv", index=True)

In [234]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2888 entries, 2012-01-01 00:00:00+01:00 to 2019-11-27 00:00:00+01:00
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   suction_tension (20cm)   2888 non-null   float64
 1   suction_tension (40cm)   2888 non-null   float64
 2   suction_tension (70cm)   2888 non-null   float64
 3   water_temperature        2888 non-null   float64
 4   barometric_pressure_qfe  2888 non-null   float64
 5   dew_point                2888 non-null   float64
 6   water_level              2888 non-null   float64
 7   inflow_linth (m3/s)      2888 non-null   float64
 8   outflow_limmat (m3/s)    2888 non-null   float64
 9   precipitation (mm)       2888 non-null   float64
 10  change_rate              2887 non-null   float64
 11  Evapotranspiration (mm)  0 non-null      object 
dtypes: float64(11), object(1)
memory usage: 357.9+ KB
